# Model Save and Load

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

np.random.seed(7777)
tf.random.set_seed(7777)

In [2]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(16, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)
    
    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(32, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])
    
    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 16)   448         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 8, 8, 16)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 32)     544         ['max_pooling2d[0][0]']          
                                                                                             

In [5]:
lr = 0.03
opt = tf.keras.optimizers.Adam(lr)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

## 1. save 함수
- 모델을 통째로 저장
- 파일만 있으면 모델을 복원할 수 있음
- 대신 용량이 큼

In [6]:
model.save("checkpoints/sample/model.h5")  # h5 : 모델 확장자 중 가장 많이 사용됨

In [8]:
model_loaded = tf.keras.models.load_model("checkpoints/sample/model.h5")

model_loaded.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 16)   448         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 8, 8, 16)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 32)     544         ['max_pooling2d[0][0]']          
                                                                                             

## 2.  save_weights 함수 

 - weights만 저장 하므로, 저장공간이 절약됨. 
 - 대신 `build_resnet` 같은 모델 정보 함수가 있어야 모델 복원 가능

In [9]:
model.save_weights("checkpoints/sample/model.h5")

In [10]:
new_model = build_resnet((32, 32, 3))
new_model.load_weights("checkpoints/sample/model.h5")

In [12]:
new_model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_8 (Conv2D)              (None, 16, 16, 16)   448         ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 8, 8, 16)    0           ['conv2d_8[0][0]']               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 8, 8, 32)     544         ['max_pooling2d_2[0][0]']        
                                                                                             

## 3. Callbacks 함수 사용하기
- 매 epoch 마다 모델을 저장

In [14]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [15]:
save_path = 'checkpoints/{epoch:02d}-{val_loss:.2f}.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, 
                                                monitor='val_accuracy', 
                                                save_best_only=True)

- save_best_only=True : 이전 epoch에 비해 성능이 좋은 epoch인 경우에만 저장
- 항상 True 인게 좋은것은 아님!

In [16]:
model.fit(x=train_x, 
          y=train_y, 
          epochs=1,
          validation_data=(test_x, test_y), 
          callbacks=[checkpoint])

1563/1563 [==============================] - 37s 22ms/step - loss: 2.1128 - accuracy: 0.2137 - val_loss: 1.9618 - val_accuracy: 0.2773


## 4. pb형식으로 저장
 - 모델을 protoBuffer 형식으로 저장 (h5 형식이 아니다!)
 - 모델 구조와 weight를 함께 저장

In [17]:
save_path = 'checkpoints/{epoch:02d}-{val_loss:.2f}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, 
                                                monitor='val_accuracy', 
                                                save_best_only=True)

In [18]:
model.fit(x=train_x, 
          y=train_y, 
          epochs=1,
          validation_data=(test_x, test_y), 
          callbacks=[checkpoint])

1562/1563 [============================>.] - ETA: 0s - loss: 1.9394 - accuracy: 0.2918

INFO:tensorflow:Assets written to: checkpoints\01-1.95\assets


INFO:tensorflow:Assets written to: checkpoints\01-1.95\assets


1563/1563 [==============================] - 28s 18ms/step - loss: 1.9394 - accuracy: 0.2918 - val_loss: 1.9543 - val_accuracy: 0.2725


In [22]:
model_pb = tf.saved_model.load("checkpoints/01-1.95")